# Fig. AR Climatology

Plot domain and variables used for EOF analysis

In [ ]:
# Import Python modules
import os, sys
import numpy as np
import pandas as  pd
import xarray as xr
# plotting
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import AxesGrid
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
import seaborn as sns
import cmocean.cm as cmo
import cmocean
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter


from pathlib import Path
from IPython.display import Image, display
from scipy.ndimage import gaussian_filter    # smoothing contour lines

# Path to modules
sys.path.append('../modules')

# Import my modules
from plotter import draw_basemap, add_subregion_boxes, make_cmap
from ar_funcs import preprocess_ar_area_subregions

### Set up paths

In [ ]:
# Set up paths

path_to_data = '/home/nash/DATA/data/'                            # project data -- read only
path_to_out  = '/home/nash/DATA/repositories/AR_types/out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '/home/nash/DATA/repositories/AR_types/figs/'      # figures


In [ ]:
# Set a default font for all matplotlib text (can only set this ONCE; must restart kernel to change it)
rcParams['font.family'] = 'sans-serif'   # set the default font family to 'sans-serif'
rcParams['font.sans-serif'] = 'Arial'    # set the default sans-serif font to 'Arial'


In [ ]:
# label strings

ivt_units = 'kg m$^{-1}$ s$^{-1}$'

## Data

### 1) AR time series

In [ ]:
# read netCDF with fraction of area AR covers each subregion
# filename = path_to_data + 'CH1_generated_data/ar_catalog_v3_ERAI_fraction_HASIAsubregions.nc'
filename = path_to_data + 'CH1_generated_data/ar_catalog_v3_ERAI_fraction_HASIAsubregions.nc'
ds = xr.open_dataset(filename)

# Set dates
ds = ds.sel(time=slice('1979-01-01', '2018-12-31'))

## Preprocess AR subregions - get dataframe of AR days based on area threshold
df = preprocess_ar_area_subregions(df=ds.to_dataframe(), thres=0.3)
# Show table
df.head()


### 2) ERA reanalysis

In [ ]:
# Select lat/lon grid
lonmin = 45
lonmax = 110
latmin = 0
latmax = 50

### ERA5 DATA ###
def preprocess(ds):
    '''keep only selected lats and lons'''
    return ds.sel(latitude=slice(latmax, latmin), longitude=slice(lonmin, lonmax))

# open IVT anomaly data
filepath_pattern = path_to_data + 'ERA5/ivt/daily/out.era5*.nc'
f1 = xr.open_mfdataset(filepath_pattern, preprocess=preprocess, combine='by_coords')
print('ds size in GB {:0.2f}\n'.format(f1.nbytes / 1e9))

# Create new dataset to rename lat lon
era = xr.Dataset({'ivte': (['time', 'lat', 'lon'], f1['p71.162'].values),
                  'ivtn': (['time', 'lat', 'lon'], f1['p72.162'].values)},
                  coords={'time': (['time'], f1.time.values),
                          'lat': (['lat'], f1.latitude.values),
                          'lon': (['lon'], f1.longitude.values)})

era

### Select data subset

In [ ]:
# Add AR time series to era5; set as coordinate variables
era['ar'] = ('time', df.ar)
era = era.set_coords('ar')

era['location'] = ('time', df.location)
era = era.set_coords('location')

# print dataset
print(era)

In [ ]:
# Trim date range
start_date = '1979-12-01'
end_date = '2018-02-28'
idx = slice(start_date, end_date)
era = era.sel(time=idx)

# # Select DJF months
# idx = (era.time.dt.month >= 12) | (era.time.dt.month <= 2)
# era = era.sel(time=idx)

# Select AR days
idx = (era.ar >= 1)
era_ar = era.sel(time=idx)

# print results
print(era_ar)

## Calculate climatology

In [ ]:
# Mean IVT of AR days per season
# era_ar_clim = era_ar.mean(dim='time')
era_ar_clim = era_ar.groupby('time.season').mean(dim='time')

# Mean IVT of all days
era_clim = era.groupby('time.season').mean(dim='time')

In [ ]:
ar_filename = path_to_data + 'ar_catalog/globalARcatalog_ERA-Interim_1979-2019_v3.0.nc'
ar_ds = xr.open_dataset(ar_filename)
ar_ds = ar_ds.sel(lat=slice(latmin, latmax), lon=slice(lonmin, lonmax))
ar_ds = ar_ds.shape

# count number of AR time steps per season
count_DJF = len(ar_ds.groupby('time.season').groups['DJF'])
count_MAM = len(ar_ds.groupby('time.season').groups['MAM'])
count_JJA = len(ar_ds.groupby('time.season').groups['JJA'])
count_SON = len(ar_ds.groupby('time.season').groups['SON'])

count_lst = [count_DJF, count_MAM, count_JJA, count_SON]

# count number of AR time steps per gridcell
ar_da = ar_ds.groupby('time.season').count(dim='time')

## Create figure

In [ ]:
# Set up projection
mapcrs = ccrs.PlateCarree()
datacrs = ccrs.PlateCarree()

# Set tick/grid locations
dx = np.arange(lonmin,lonmax+20,20)
dy = np.arange(latmin,latmax+20,20)

# subregion info 
# [ymin, xmin]
sr_xy = [[65, 30], [75, 25], [85, 20]]
# width of subregion
sr_width = [10, 10, 10]
# height of subregion
sr_height = [10, 10, 10]


In [ ]:
# Data for plotting
data = era_clim
lons = data.lon.values
lats = data.lat.values
count_lst = [count_DJF, count_MAM, count_JJA, count_SON]

# Create figure
fig = plt.figure(figsize=(10,11))
nrows = 2
ncols = 2

# Set up Axes Grid
axes_class = (GeoAxes,dict(map_projection=mapcrs))
axgr = AxesGrid(fig, 111, axes_class=axes_class,
                nrows_ncols=(nrows, ncols), axes_pad = 0.45,
                cbar_location='bottom', cbar_mode='single',
                cbar_pad=0.05, cbar_size='3%',label_mode='')

# cmap setup
cmap = cmo.deep
clipcmap = cmocean.tools.crop_by_percent(cmap, 5, which='both', N=None)
cflevs = np.arange(125, 1000, 125)
ssn_lst = ['DJF', 'MAM', 'JJA', 'SON']

# Loop for drawing each plot
for k, (ax, ssn) in enumerate(zip(axgr, ssn_lst)):
    ax = draw_basemap(ax, extent=[lonmin,lonmax,latmin,latmax], xticks=dx, yticks=dy)
    
    # Contour Filled
    # Average Seasonal Total
    uvec = data.ivte.sel(season=ssn).values
    vvec = data.ivtn.sel(season=ssn).values
    ivt = np.sqrt(uvec**2 + vvec**2)
    cf = ax.contourf(lons, lats, ivt, transform=datacrs,
                    levels=cflevs,cmap=clipcmap, extend='max')
    
    
    # Contour Lines
    # Percent Frequency
    ar_lats = ar_da.lat.values
    ar_lons = ar_da.lon.values
    ar_count = ar_da.sel(season=ssn).squeeze()
    contr = np.divide(ar_count.values, float(count_lst[k]))
    ar_smooth = gaussian_filter(contr*100, sigma=1.5)
    clevs = np.arange(2, 22, 2)
    cs = ax.contour(ar_lons, ar_lats, ar_smooth, transform=datacrs,
                    levels=clevs, colors='k', linewidths=1.1)
    kw_clabels = {'fontsize': 9, 'inline': True, 'inline_spacing': 8, 'fmt': '%d',
                  'rightside_up': True, 'use_clabeltext': True}
    plt.clabel(cs, **kw_clabels)
    
    # IVT vectors
    Q = ax.quiver(lons, lats, uvec, vvec, transform=datacrs,
                  color='black', regrid_shape=20, pivot='middle', 
                  angles='xy', scale_units='xy', scale=50, units='xy')
    
    # quiver key
    qk = ax.quiverkey(Q, 0.7, 1.04, 250, r'250 kg m$^{-1}$ s$^{-1}$', labelpos='E',
                   coordinates='axes', fontproperties={'size': 10.0})
    
    # add subregions
    add_subregion_boxes(ax=ax, subregion_xy=sr_xy, width=sr_width, 
                        height=sr_height, ecolor='red', datacrs=datacrs)
    
    # Subtitles
    ax.set_title(ssn_lst[k], loc='left', fontsize=13)

# Colorbar (single)
cb = fig.colorbar(cf, axgr.cbar_axes[0], orientation='horizontal', drawedges=True)
cb.set_label('IVT (kg m$^{-1}$ s$^{-1}$)', fontsize=11)
cb.ax.tick_params(labelsize=10)
    
# Save figure
filepath = path_to_figs + 'ivt_clim_all_ssn' + '.png'
plt.savefig(filepath, dpi=300, bbox_inches='tight')

# Show
plt.show()

In [ ]:
# Data for plotting
data = era_clim
lons = data.lon.values
lats = data.lat.values
count_lst = [count_DJF, count_MAM, count_JJA, count_SON]

# Create figure
fig = plt.figure(figsize=(10,11))
nrows = 1
ncols = 2

# Set up Axes Grid
axes_class = (GeoAxes,dict(map_projection=mapcrs))
axgr = AxesGrid(fig, 111, axes_class=axes_class,
                nrows_ncols=(nrows, ncols), axes_pad = 0.45,
                cbar_location='bottom', cbar_mode='single',
                cbar_pad=0.05, cbar_size='3%',label_mode='')

# cmap setup
cmap = cmo.deep
clipcmap = cmocean.tools.crop_by_percent(cmap, 5, which='both', N=None)
cflevs = np.arange(125, 625, 125)
ssn_lst = ['DJF', 'MAM']

# Loop for drawing each plot
for k, (ax, ssn) in enumerate(zip(axgr, ssn_lst)):
    ax = draw_basemap(ax, extent=[lonmin,lonmax,latmin,latmax], xticks=dx, yticks=dy)
    
    # Contour Filled
    # Average Seasonal Total
    uvec = data.ivte.sel(season=ssn).values
    vvec = data.ivtn.sel(season=ssn).values
    ivt = np.sqrt(uvec**2 + vvec**2)
    cf = ax.contourf(lons, lats, ivt, transform=datacrs,
                    levels=cflevs,cmap=clipcmap, extend='max')
    
    
    # Contour Lines
    # Percent Frequency
    ar_lats = ar_da.lat.values
    ar_lons = ar_da.lon.values
    ar_count = ar_da.sel(season=ssn).squeeze()
    contr = np.divide(ar_count.values, float(count_lst[k]))
    ar_smooth = gaussian_filter(contr*100, sigma=1.5)
    clevs = np.arange(2, 22, 2)
    cs = ax.contour(ar_lons, ar_lats, ar_smooth, transform=datacrs,
                    levels=clevs, colors='k', linewidths=1.1)
    kw_clabels = {'fontsize': 9, 'inline': True, 'inline_spacing': 8, 'fmt': '%d',
                  'rightside_up': True, 'use_clabeltext': True}
    plt.clabel(cs, **kw_clabels)
    
    # IVT vectors
    Q = ax.quiver(lons, lats, uvec, vvec, transform=datacrs,
                  color='black', regrid_shape=20, pivot='middle', 
                  angles='xy', scale_units='xy', scale=50, units='xy')
    
    # quiver key
    qk = ax.quiverkey(Q, 0.7, 1.04, 250, r'250 kg m$^{-1}$ s$^{-1}$', labelpos='E',
                   coordinates='axes', fontproperties={'size': 10.0})
    
    # add subregions
    add_subregion_boxes(ax=ax, subregion_xy=sr_xy, width=sr_width, 
                        height=sr_height, ecolor='red', datacrs=datacrs)
    
    # Subtitles
    ax.set_title(ssn_lst[k], loc='left', fontsize=13)

# Colorbar (single)
cb = fig.colorbar(cf, axgr.cbar_axes[0], orientation='horizontal', drawedges=True)
cb.set_label('IVT (kg m$^{-1}$ s$^{-1}$)', fontsize=11)
cb.ax.tick_params(labelsize=10)
    
# Save figure
filepath = path_to_figs + 'ivt_clim_djf_mam' + '.png'
plt.savefig(filepath, dpi=300, bbox_inches='tight')

# Show
plt.show()